In [10]:
import json
import numpy as np
import pandas as pd

def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f, 1):
            try:
                line = line.strip()
                if line:
                    json_obj = json.loads(line)
                    data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Error parsing line {line_number} in {file_path}: {e}")
            except Exception as e:
                print(f"Unexpected error on line {line_number} in {file_path}: {e}")
    return data

def process_data(data, model_name):
    processed = {}
    for item in data:
        dataset = item['dataset'].split('/')[-1]
        metrics = item['metrics']
        if 'acc1' in metrics:
            processed[f"{dataset}_acc1"] = metrics['acc1']
        if 'acc5' in metrics:
            processed[f"{dataset}_acc5"] = metrics['acc5']
    return {model_name: processed}

def calculate_average_scores(all_data, models):
    """
    Calculate the average acc1 and acc5 scores for each model.
    """
    average_scores = {}
    
    for model in models:
        acc1_scores = []
        acc5_scores = []
        
        if model in all_data:
            for metric, score in all_data[model].items():
                if "acc1" in metric:
                    acc1_scores.append(score)
                if "acc5" in metric:
                    acc5_scores.append(score)
                    
        # Calculate averages if scores are available
        avg_acc1 = np.mean(acc1_scores) if acc1_scores else None
        avg_acc5 = np.mean(acc5_scores) if acc5_scores else None
        
        average_scores[model] = {
            'avg_acc1': avg_acc1,
            'avg_acc5': avg_acc5
        }
    
    return average_scores

# Load and process the data
models = ['mobilemclip_s1_6m_0_1', 'mobilemclip_s1_6m_025_075']
all_data = {}

for model in models:
    file_path = f'results/{model}.jsonl'
    try:
        data = load_data(file_path)
        all_data.update(process_data(data, model))
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Calculate average scores and display them
average_scores = calculate_average_scores(all_data, models)
average_scores_df = pd.DataFrame(average_scores).T

print(average_scores_df)

                           avg_acc1  avg_acc5
mobilemclip_s1_6m_0_1      0.186394       NaN
mobilemclip_s1_6m_025_075  0.176658       NaN


In [11]:
def process_data_with_nan(data, model_name):
    """
    Process the data, handling NaN values by treating them as None.
    """
    processed = {}
    for item in data:
        dataset = item['dataset'].split('/')[-1]
        metrics = item['metrics']

        acc1 = metrics.get('acc1', None)
        acc5 = metrics.get('acc5', None)

        # Handle NaN values explicitly, setting them to None
        if isinstance(acc5, float) and np.isnan(acc5):
            acc5 = None

        if acc1 is not None:
            processed[f"{dataset}_acc1"] = acc1
        if acc5 is not None:
            processed[f"{dataset}_acc5"] = acc5

    return {model_name: processed}

# Modify the main loading and processing logic to use the new function
all_data = {}

for model in models:
    file_path = f'results/{model}.jsonl'
    try:
        data = load_data(file_path)
        all_data.update(process_data_with_nan(data, model))
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Recalculate average scores and display them
average_scores = calculate_average_scores(all_data, models)
average_scores_df = pd.DataFrame(average_scores).T

print(average_scores_df)

                           avg_acc1  avg_acc5
mobilemclip_s1_6m_0_1      0.186394  0.424934
mobilemclip_s1_6m_025_075  0.176658  0.431909


In [15]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f, 1):
            try:
                line = line.strip()
                if line:
                    json_obj = json.loads(line)
                    data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Error parsing line {line_number} in {file_path}: {e}")
            except Exception as e:
                print(f"Unexpected error on line {line_number} in {file_path}: {e}")
    return data

def process_data(data, model_name, selected_datasets):
    processed = {
        'zeroshot_cls': {},
        'flickr30k_ret': {},
        'coco_ret': {},
        'avg_perf': 0
    }
    dataset_scores = []

    for item in data:
        dataset = item['dataset'].split('/')[-1]
        metrics = item['metrics']

        if dataset == 'imagenet1k':
            processed['zeroshot_cls']['IN-val'] = metrics.get('acc5', 0)
        elif dataset in ['imagenetv2', 'imagenet-r', 'imagenet-o', 'imagenet_sketch']:
            processed['zeroshot_cls'][dataset] = metrics.get('acc5', 0)
        elif dataset == "flickr30k":
            processed['flickr30k_ret']['I2T'] = metrics.get('image_retrieval_recall@5', 0)
            processed['flickr30k_ret']['T2I'] = metrics.get('text_retrieval_recall@5', 0)
        elif dataset == "mscoco_captions":
            processed['coco_ret']['I2T'] = metrics.get('image_retrieval_recall@5', 0)
            processed['coco_ret']['T2I'] = metrics.get('text_retrieval_recall@5', 0)

        if dataset in selected_datasets:
            if 'acc5' in metrics:
                dataset_scores.append(metrics['acc5'])
            # recall@1은 flickr30k, coco_captions 에서만 사용되므로, 다른 데이터셋은 제외
            elif dataset in ["flickr30k", "coco_captions"] and any(k.startswith('recall@1') for k in metrics):
                # recall@1_image2text 또는 recall@1_text2image 중 하나라도 있으면 사용
                dataset_scores.append(metrics.get('image_retrieval_recall@5', metrics.get('text_retrieval_recall@5', 0)))


    if dataset_scores:
        processed['avg_perf'] = np.mean(dataset_scores)

    return {model_name: processed}


def visualize_retrieval(all_data, models):
    df_data = {}
    for model_name, model_data in all_data.items():
        row = {
            'Model': model_name,
            'IN-val': model_data['zeroshot_cls'].get('IN-val', 0),
            'IN-shift': np.mean([v for k, v in model_data['zeroshot_cls'].items() if k != 'IN-val']),
            'Flickr30k T→I': model_data['flickr30k_ret'].get('I2T', 0),
            'Flickr30k I→T': model_data['flickr30k_ret'].get('T2I', 0),
            'COCO T→I': model_data['coco_ret'].get('I2T', 0),
            'COCO I→T': model_data['coco_ret'].get('T2I', 0),
            'Avg. Perf. on 23': model_data['avg_perf']
        }
        df_data[model_name] = row
    df = pd.DataFrame.from_dict(df_data, orient='index')
    print(df)
    return df

# --- 메인 실행 부분 ---
models = ['mobilemclip_s1_6m_1_0', 'mobilemclip_s1_6m_025_075', 'mobilemclip_s1_6m_075_025', 'mobilemclip_s1_6m_0_1']
all_data = {}

selected_datasets = [
    "objectnet", "fer2013", "voc2007", "sun397", "cars", "mnist", "stl10", "gtsrb",
    "cifar10", "cifar100", "imagenet1k", "pets", "clevr_closest_object_distance",
    "caltech101", "svhn", "dmlab", "eurosat", "diabetic_retinopathy", "resisc45",
    "imagenetv2", "imagenet_sketch", "imagenet-r", "imagenet-o"
]

for model in models:
    file_path = f'results/{model}.jsonl'
    try:
        data = load_data(file_path)
        all_data.update(process_data(data, model, selected_datasets))
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

df = visualize_retrieval(all_data, models)

                                               Model   IN-val  IN-shift  \
mobilemclip_s1_6m_1_0          mobilemclip_s1_6m_1_0  0.18320  0.239230   
mobilemclip_s1_6m_025_075  mobilemclip_s1_6m_025_075  0.20142  0.252474   

                           Flickr30k T→I  Flickr30k I→T  COCO T→I  COCO I→T  \
mobilemclip_s1_6m_1_0             0.2096          0.279  0.139384    0.1880   
mobilemclip_s1_6m_025_075         0.2272          0.260  0.151819    0.2044   

                           Avg. Perf. on 23  
mobilemclip_s1_6m_1_0              0.508060  
mobilemclip_s1_6m_025_075          0.523343  
